In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,3"


/home/gunala/miniconda3/envs/DialGenEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# a couple adjustments made bc i only have 2 GPUs right now;
# --> float16 instead of float32, device='balanced' instead of auto
model_path = "/home/gunala/LLMDialGen/llama_model_7b_chat"
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="balanced").eval()
use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side='left', use_fast=use_fast_tokenizer)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.97s/it]


In [3]:
LLAMA2_PROMPT = " <s>[INST] <<SYS>> \
                {{  }} \
                <</SYS>> \
                {{  }} [/INST] "        

In [4]:
# generate the llama response provided   
def generate_llama_response(system_prompt, user_prompt):
    # generate individual response; assuming system prompt is staying the same
    prompt_formatted = LLAMA2_PROMPT.format(system_prompt,user_prompt)
    input_ids = tokenizer(prompt_formatted, padding=True, return_tensors="pt")
    input_ids['input_ids'] = input_ids['input_ids'].cuda()#.cuda()
    input_ids['attention_mask'] = input_ids['attention_mask'].cuda()#.cuda()
    num_input_tokens = input_ids['input_ids'].shape[1]
    outputs = model.generate(input_ids['input_ids'], attention_mask=input_ids['attention_mask'].half(),
                                max_new_tokens=64, do_sample=True, pad_token_id=tokenizer.pad_token_id)
    generation = tokenizer.batch_decode(outputs[:, num_input_tokens:], skip_special_tokens=True)

    return generation

In [5]:
# unit gen
print(generate_llama_response('you are a healthy individual playing prisoners dilemma' ,'i would like to defect this round'))

['I\'m not sure I understand what you are saying with "{ }". Could you explain?']
